In [1]:
import random
import tensorflow as tf
import tensorflow.keras.backend as K

import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score
from tensorflow.keras.utils import to_categorical

In [2]:
tf.keras.backend.clear_session()
np.random.seed(2020) # NumPy
random.seed(2020) # Python
tf.random.set_seed(2020) # Tensorflow

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
# input files provided in https://www.kaggle.com/c/ieee-fraud-detection/
train = pd.read_csv('input/train_transaction.csv').drop(['TransactionID', 'TransactionDT'], axis=1)

<h5>Category conversion</h5>

In [10]:
'''
Data description as provided in https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203

TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
TransactionAMT: transaction payment amount in USD
ProductCD: product code, the product for each transaction
card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
addr: address
dist: distance
P_ and (R__) emaildomain: purchaser and recipient email domain
C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
D1-D15: timedelta, such as days between previous transaction, etc.
M1-M9: match, such as names on card and address, etc.
Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.
'''

'\nData description as provided in https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203\n\nTransactionDT: timedelta from a given reference datetime (not an actual timestamp)\nTransactionAMT: transaction payment amount in USD\nProductCD: product code, the product for each transaction\ncard1 - card6: payment card information, such as card type, card category, issue bank, country, etc.\naddr: address\ndist: distance\nP_ and (R__) emaildomain: purchaser and recipient email domain\nC1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.\nD1-D15: timedelta, such as days between previous transaction, etc.\nM1-M9: match, such as names on card and address, etc.\nVxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.\n'

In [6]:
# target
target_col = 'isFraud'

# using the columns chosen by the model developer
# removing card1 since it is ordinally encoded but has a high number of categories although it has a high feature importance
# including TransactionAmt in order to work with the counterfactuals later
# since DiCE works with one-hot encoding only the number of columns will increase too much
useful_features = list(train.iloc[:, 1:53].columns)
useful_features.append(target_col)
useful_features.remove('card1')

train = train[useful_features]

In [7]:
# categorical features
cat_columns = ['P_emaildomain', 'R_emaildomain', 'addr1', 'addr2', 'ProductCD']

for col in train.columns:
    if col.startswith('card') or col.startswith('M'):
        cat_columns.append(col)

# select only the columns that are not objects
cat_object_columns = []
for col in cat_columns:
    if train[col].dtypes == type(object):
        cat_object_columns.append(col)
    elif col in ['addr1', 'addr2', 'card2', 'card3', 'card5']:
        train[col] = train[col].astype('str')
        cat_object_columns.append(col)

In [8]:
cat_vars_ord = {}
columns = train.columns.tolist()
for col in range(train.shape[1]):
    if columns[col] in cat_columns:
        cat_vars_ord[col] = len(train[columns[col]].unique())

<h5>Retrieving the training, test and validation datasets</h5>

In [9]:
split_train_validation = int(train.shape[0]*0.8)
train[cat_object_columns] = train[cat_object_columns].fillna('0')

In [10]:
# one-hot encoding for DiCE
train_dice = pd.get_dummies(train, columns=cat_columns, drop_first=False)
X_train_dice = train_dice.drop(target_col, axis=1).iloc[:split_train_validation].fillna(0)
X_val_dice = train_dice.drop(target_col, axis=1).iloc[split_train_validation:].fillna(0)

In [11]:
# ordinal encoding for Alibi
enc = OrdinalEncoder().fit(train[cat_object_columns])
train[cat_object_columns] = enc.transform(train[cat_object_columns])

X_train_alibi = train.drop(target_col, axis=1).iloc[:split_train_validation].fillna(0)
X_val_alibi = train.drop(target_col, axis=1).iloc[split_train_validation:].fillna(0)

In [12]:
X_train_alibi.head()

,TransactionAmt,ProductCD,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,68.5,4.0,500.0,42.0,2.0,38.0,2.0,166.0,65.0,19.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0,1.0,2.0,0.0,0.0,0.0
1,29.0,4.0,303.0,42.0,3.0,2.0,2.0,173.0,65.0,0.0,0.0,17.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0
2,59.0,4.0,389.0,42.0,4.0,58.0,3.0,178.0,65.0,287.0,0.0,36.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315.0,0.0,0.0,0.0,315.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
3,50.0,4.0,466.0,42.0,3.0,14.0,3.0,282.0,65.0,0.0,0.0,54.0,0.0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0,0.0,111.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0
4,50.0,1.0,413.0,42.0,3.0,2.0,2.0,241.0,65.0,0.0,0.0,17.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# separating the target columns that will be used in both datasets
y_train = train[target_col].iloc[:split_train_validation].fillna(0)
y_val = train[target_col].iloc[split_train_validation:].fillna(0)

<h5>Creating feature metadata to be used by DiCE</h5>

In [14]:
features = {}

# categorical values
for col in cat_columns:
    # sorting values
    values = train[col].dropna().unique()
    values.sort()
    
    if col in cat_object_columns:
        features[col] = enc.categories_[cat_object_columns.index(col)]
    else:
        features[col] = [int(round(values.min())), int(round(values.max()))]
        features[col] = [str(v) for v in values]
        
# numerical values
numerical_columns = ['TransactionAmt', 'dist1', 'dist2']

# integer columns
integer_columns = []

# appending other numerical features
for col in train.columns:
    if col.startswith('V') or col.startswith('D') or col.startswith('C'):
        numerical_columns.append(col)

for col in numerical_columns:
    features[col] = [int(round(train[col].min())), int(round(train[col].max()))]

<h5>Training the models</h5>

In [15]:
def create_model(num_attributes, loss_fn, return_proba=False, show_summary=False):
    metrics = [
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
    ]
    
    inps = Input(shape=(num_attributes,))
    x = Dense(512, activation='relu')(inps)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    if return_proba:
        x = Dense(2, activation='softmax')(x)
    else:
        x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inps, outputs=x)
    model.compile(optimizer=Adam(), loss=[loss_fn], metrics=metrics)
    
    if show_summary:
        model.summary()
        
    return model

In [16]:
weight_for_0 = (1 / y_train[y_train==0].shape[0])*(y_train.shape[0])/2.0 
weight_for_1 = (1 / y_train[y_train==1].shape[0])*(y_train.shape[0])/2.0

<p>Model to be used by DiCE</p>

In [17]:
model_clf = create_model(X_train_dice.shape[1], 'binary_crossentropy', False, True)
model_clf.fit(X_train_dice.values, y_train, epochs=500, batch_size=512, validation_data=(X_val_dice.values, y_val), verbose=True, class_weight={1:weight_for_1, 0:weight_for_0}, use_multiprocessing=True)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1340)]            0         
_________________________________________________________________
dense (Dense)                (None, 512)               686592    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)              

Epoch 18/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.3583 - tp: 13572.0000 - fp: 61229.0000 - tn: 394604.0000 - fn: 3027.0000 - accuracy: 0.8640 - precision: 0.1814 - recall: 0.8176 - auc: 0.9191 - val_loss: 0.3121 - val_tp: 2741.0000 - val_fp: 11868.0000 - val_tn: 102176.0000 - val_fn: 1323.0000 - val_accuracy: 0.8883 - val_precision: 0.1876 - val_recall: 0.6745 - val_auc: 0.8668
Epoch 19/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.3558 - tp: 13553.0000 - fp: 61119.0000 - tn: 394714.0000 - fn: 3046.0000 - accuracy: 0.8642 - precision: 0.1815 - recall: 0.8165 - auc: 0.9206 - val_loss: 0.2781 - val_tp: 2690.0000 - val_fp: 10270.0000 - val_tn: 103774.0000 - val_fn: 1374.0000 - val_accuracy: 0.9014 - val_precision: 0.2076 - val_recall: 0.6619 - val_auc: 0.8690
Epoch 20/2000
923/923 [==============================] - 6s 7ms/step - loss: 0.3530 - tp: 13626.0000 - fp: 59733.0000 - tn: 396100.0000 - fn: 2973.0000 - accuracy: 0.8673 - precisi

923/923 [==============================] - 7s 7ms/step - loss: 0.3083 - tp: 14101.0000 - fp: 53891.0000 - tn: 401942.0000 - fn: 2498.0000 - accuracy: 0.8806 - precision: 0.2074 - recall: 0.8495 - auc: 0.9411 - val_loss: 0.2755 - val_tp: 2728.0000 - val_fp: 9882.0000 - val_tn: 104162.0000 - val_fn: 1336.0000 - val_accuracy: 0.9050 - val_precision: 0.2163 - val_recall: 0.6713 - val_auc: 0.8743
Epoch 39/2000
923/923 [==============================] - 6s 7ms/step - loss: 0.3065 - tp: 14090.0000 - fp: 53370.0000 - tn: 402463.0000 - fn: 2509.0000 - accuracy: 0.8817 - precision: 0.2089 - recall: 0.8488 - auc: 0.9418 - val_loss: 0.2585 - val_tp: 2622.0000 - val_fp: 9214.0000 - val_tn: 104830.0000 - val_fn: 1442.0000 - val_accuracy: 0.9098 - val_precision: 0.2215 - val_recall: 0.6452 - val_auc: 0.8658
Epoch 40/2000
923/923 [==============================] - 6s 7ms/step - loss: 0.3025 - tp: 14101.0000 - fp: 51624.0000 - tn: 404209.0000 - fn: 2498.0000 - accuracy: 0.8854 - precision: 0.2145 - rec

923/923 [==============================] - 6s 7ms/step - loss: 0.2481 - tp: 14706.0000 - fp: 44839.0000 - tn: 410994.0000 - fn: 1893.0000 - accuracy: 0.9011 - precision: 0.2470 - recall: 0.8860 - auc: 0.9623 - val_loss: 0.2513 - val_tp: 2619.0000 - val_fp: 8725.0000 - val_tn: 105319.0000 - val_fn: 1445.0000 - val_accuracy: 0.9139 - val_precision: 0.2309 - val_recall: 0.6444 - val_auc: 0.8535
Epoch 79/2000
923/923 [==============================] - 7s 7ms/step - loss: 0.2481 - tp: 14704.0000 - fp: 45024.0000 - tn: 410809.0000 - fn: 1895.0000 - accuracy: 0.9007 - precision: 0.2462 - recall: 0.8858 - auc: 0.9624 - val_loss: 0.2156 - val_tp: 2527.0000 - val_fp: 7225.0000 - val_tn: 106819.0000 - val_fn: 1537.0000 - val_accuracy: 0.9258 - val_precision: 0.2591 - val_recall: 0.6218 - val_auc: 0.8683
Epoch 80/2000
923/923 [==============================] - 6s 7ms/step - loss: 0.2455 - tp: 14729.0000 - fp: 43941.0000 - tn: 411892.0000 - fn: 1870.0000 - accuracy: 0.9030 - precision: 0.2510 - rec

923/923 [==============================] - 7s 7ms/step - loss: 0.2139 - tp: 15049.0000 - fp: 39572.0000 - tn: 416261.0000 - fn: 1550.0000 - accuracy: 0.9130 - precision: 0.2755 - recall: 0.9066 - auc: 0.9721 - val_loss: 0.2181 - val_tp: 2498.0000 - val_fp: 7434.0000 - val_tn: 106610.0000 - val_fn: 1566.0000 - val_accuracy: 0.9238 - val_precision: 0.2515 - val_recall: 0.6147 - val_auc: 0.8570
Epoch 119/2000
923/923 [==============================] - 7s 7ms/step - loss: 0.2130 - tp: 15024.0000 - fp: 39304.0000 - tn: 416529.0000 - fn: 1575.0000 - accuracy: 0.9135 - precision: 0.2765 - recall: 0.9051 - auc: 0.9723 - val_loss: 0.2170 - val_tp: 2529.0000 - val_fp: 7442.0000 - val_tn: 106602.0000 - val_fn: 1535.0000 - val_accuracy: 0.9240 - val_precision: 0.2536 - val_recall: 0.6223 - val_auc: 0.8608
Epoch 120/2000
923/923 [==============================] - 6s 7ms/step - loss: 0.2123 - tp: 15046.0000 - fp: 39267.0000 - tn: 416566.0000 - fn: 1553.0000 - accuracy: 0.9136 - precision: 0.2770 - r

...

923/923 [==============================] - 6s 6ms/step - loss: 0.0740 - tp: 16192.0000 - fp: 14030.0000 - tn: 441803.0000 - fn: 407.0000 - accuracy: 0.9694 - precision: 0.5358 - recall: 0.9755 - auc: 0.9960 - val_loss: 0.2647 - val_tp: 2173.0000 - val_fp: 3186.0000 - val_tn: 110858.0000 - val_fn: 1891.0000 - val_accuracy: 0.9570 - val_precision: 0.4055 - val_recall: 0.5347 - val_auc: 0.8394
Epoch 1919/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0737 - tp: 16188.0000 - fp: 13946.0000 - tn: 441887.0000 - fn: 411.0000 - accuracy: 0.9696 - precision: 0.5372 - recall: 0.9752 - auc: 0.9962 - val_loss: 0.2411 - val_tp: 2108.0000 - val_fp: 2879.0000 - val_tn: 111165.0000 - val_fn: 1956.0000 - val_accuracy: 0.9591 - val_precision: 0.4227 - val_recall: 0.5187 - val_auc: 0.8383
Epoch 1920/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0719 - tp: 16204.0000 - fp: 13620.0000 - tn: 442213.0000 - fn: 395.0000 - accuracy: 0.9703 - precision: 0.5433 - re

923/923 [==============================] - 6s 6ms/step - loss: 0.0720 - tp: 16199.0000 - fp: 13892.0000 - tn: 441941.0000 - fn: 400.0000 - accuracy: 0.9697 - precision: 0.5383 - recall: 0.9759 - auc: 0.9964 - val_loss: 0.2546 - val_tp: 2225.0000 - val_fp: 3866.0000 - val_tn: 110178.0000 - val_fn: 1839.0000 - val_accuracy: 0.9517 - val_precision: 0.3653 - val_recall: 0.5475 - val_auc: 0.8451
Epoch 1939/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0734 - tp: 16177.0000 - fp: 14029.0000 - tn: 441804.0000 - fn: 422.0000 - accuracy: 0.9694 - precision: 0.5356 - recall: 0.9746 - auc: 0.9961 - val_loss: 0.2539 - val_tp: 2297.0000 - val_fp: 4057.0000 - val_tn: 109987.0000 - val_fn: 1767.0000 - val_accuracy: 0.9507 - val_precision: 0.3615 - val_recall: 0.5652 - val_auc: 0.8529
Epoch 1940/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0721 - tp: 16189.0000 - fp: 13836.0000 - tn: 441997.0000 - fn: 410.0000 - accuracy: 0.9698 - precision: 0.5392 - re

923/923 [==============================] - 6s 6ms/step - loss: 0.0769 - tp: 16172.0000 - fp: 14220.0000 - tn: 441613.0000 - fn: 427.0000 - accuracy: 0.9690 - precision: 0.5321 - recall: 0.9743 - auc: 0.9959 - val_loss: 0.2594 - val_tp: 2125.0000 - val_fp: 3001.0000 - val_tn: 111043.0000 - val_fn: 1939.0000 - val_accuracy: 0.9582 - val_precision: 0.4146 - val_recall: 0.5229 - val_auc: 0.8407
Epoch 1959/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0711 - tp: 16204.0000 - fp: 13455.0000 - tn: 442378.0000 - fn: 395.0000 - accuracy: 0.9707 - precision: 0.5463 - recall: 0.9762 - auc: 0.9963 - val_loss: 0.2633 - val_tp: 2172.0000 - val_fp: 3671.0000 - val_tn: 110373.0000 - val_fn: 1892.0000 - val_accuracy: 0.9529 - val_precision: 0.3717 - val_recall: 0.5344 - val_auc: 0.8372
Epoch 1960/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0715 - tp: 16199.0000 - fp: 13773.0000 - tn: 442060.0000 - fn: 400.0000 - accuracy: 0.9700 - precision: 0.5405 - re

923/923 [==============================] - 6s 6ms/step - loss: 0.0760 - tp: 16167.0000 - fp: 14621.0000 - tn: 441212.0000 - fn: 432.0000 - accuracy: 0.9681 - precision: 0.5251 - recall: 0.9740 - auc: 0.9959 - val_loss: 0.2231 - val_tp: 2152.0000 - val_fp: 2977.0000 - val_tn: 111067.0000 - val_fn: 1912.0000 - val_accuracy: 0.9586 - val_precision: 0.4196 - val_recall: 0.5295 - val_auc: 0.8469
Epoch 1979/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0732 - tp: 16189.0000 - fp: 14016.0000 - tn: 441817.0000 - fn: 410.0000 - accuracy: 0.9695 - precision: 0.5360 - recall: 0.9753 - auc: 0.9962 - val_loss: 0.2374 - val_tp: 2138.0000 - val_fp: 2881.0000 - val_tn: 111163.0000 - val_fn: 1926.0000 - val_accuracy: 0.9593 - val_precision: 0.4260 - val_recall: 0.5261 - val_auc: 0.8419
Epoch 1980/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0732 - tp: 16208.0000 - fp: 13708.0000 - tn: 442125.0000 - fn: 391.0000 - accuracy: 0.9702 - precision: 0.5418 - re

923/923 [==============================] - 6s 6ms/step - loss: 0.0742 - tp: 16164.0000 - fp: 14011.0000 - tn: 441822.0000 - fn: 435.0000 - accuracy: 0.9694 - precision: 0.5357 - recall: 0.9738 - auc: 0.9961 - val_loss: 0.2369 - val_tp: 2180.0000 - val_fp: 3021.0000 - val_tn: 111023.0000 - val_fn: 1884.0000 - val_accuracy: 0.9585 - val_precision: 0.4192 - val_recall: 0.5364 - val_auc: 0.8414
Epoch 1999/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0725 - tp: 16202.0000 - fp: 13853.0000 - tn: 441980.0000 - fn: 397.0000 - accuracy: 0.9698 - precision: 0.5391 - recall: 0.9761 - auc: 0.9964 - val_loss: 0.2439 - val_tp: 2122.0000 - val_fp: 3656.0000 - val_tn: 110388.0000 - val_fn: 1942.0000 - val_accuracy: 0.9526 - val_precision: 0.3673 - val_recall: 0.5221 - val_auc: 0.8329
Epoch 2000/2000
923/923 [==============================] - 6s 6ms/step - loss: 0.0746 - tp: 16179.0000 - fp: 14405.0000 - tn: 441428.0000 - fn: 420.0000 - accuracy: 0.9686 - precision: 0.5290 - re

In [18]:
model_clf.save('model_clf')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_clf\assets


<p>Model to be used by Alibi</p>

In [19]:
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
tf.executing_eagerly()

Instructions for updating:
non-resource variables are not supported in the long term


False

In [20]:
model_clf_proba = create_model(X_train_alibi.shape[1], 'binary_crossentropy', True, True)
model_clf_proba.fit(X_train_alibi.values, to_categorical(y_train), epochs=500, batch_size=512, validation_data=(X_val_alibi.values, to_categorical(y_val)), verbose=True, class_weight={1:weight_for_1, 0:weight_for_0}, use_multiprocessing=True)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 51)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               26624     
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)              

Epoch 17/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.4152 - tp: 396043.0000 - fp: 76389.0000 - tn: 396043.0000 - fn: 76389.0000 - accuracy: 0.8383 - precision: 0.8383 - recall: 0.8383 - auc: 0.9066 - val_loss: 0.3015 - val_tp: 107004.0000 - val_fp: 11104.0000 - val_tn: 107004.0000 - val_fn: 11104.0000 - val_accuracy: 0.9060 - val_precision: 0.9060 - val_recall: 0.9060 - val_auc: 0.9604
Epoch 18/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.4111 - tp: 396599.0000 - fp: 75833.0000 - tn: 396599.0000 - fn: 75833.0000 - accuracy: 0.8395 - precision: 0.8395 - recall: 0.8395 - auc: 0.9080 - val_loss: 0.2956 - val_tp: 106534.0000 - val_fp: 11574.0000 - val_tn: 106534.0000 - val_fn: 11574.0000 - val_accuracy: 0.9020 - val_precision: 0.9020 - val_recall: 0.9020 - val_auc: 0.9599
Epoch 19/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.4074 - tp: 397836.0000 - fp: 74596.0000 - tn: 397836.0000 - fn: 745

472432/472432 [==============================] - 3s 7us/sample - loss: 0.3477 - tp: 407904.0000 - fp: 64528.0000 - tn: 407904.0000 - fn: 64528.0000 - accuracy: 0.8634 - precision: 0.8634 - recall: 0.8634 - auc: 0.9316 - val_loss: 0.3262 - val_tp: 103026.0000 - val_fp: 15082.0000 - val_tn: 103026.0000 - val_fn: 15082.0000 - val_accuracy: 0.8723 - val_precision: 0.8723 - val_recall: 0.8723 - val_auc: 0.9394
Epoch 56/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.3488 - tp: 407010.0000 - fp: 65422.0000 - tn: 407010.0000 - fn: 65422.0000 - accuracy: 0.8615 - precision: 0.8615 - recall: 0.8615 - auc: 0.9308 - val_loss: 0.2749 - val_tp: 106294.0000 - val_fp: 11814.0000 - val_tn: 106294.0000 - val_fn: 11814.0000 - val_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 0.9000 - val_auc: 0.9590
Epoch 57/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.3490 - tp: 408424.0000 - fp: 64008.0000 - tn: 408424.0000 - fn: 64008.0000 - accu

472432/472432 [==============================] - 3s 7us/sample - loss: 0.3212 - tp: 412104.0000 - fp: 60328.0000 - tn: 412104.0000 - fn: 60328.0000 - accuracy: 0.8723 - precision: 0.8723 - recall: 0.8723 - auc: 0.9406 - val_loss: 0.2456 - val_tp: 107910.0000 - val_fp: 10198.0000 - val_tn: 107910.0000 - val_fn: 10198.0000 - val_accuracy: 0.9137 - val_precision: 0.9137 - val_recall: 0.9137 - val_auc: 0.9684
Epoch 94/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.3192 - tp: 411703.0000 - fp: 60729.0000 - tn: 411703.0000 - fn: 60729.0000 - accuracy: 0.8715 - precision: 0.8715 - recall: 0.8715 - auc: 0.9405 - val_loss: 0.2101 - val_tp: 109628.0000 - val_fp: 8480.0000 - val_tn: 109628.0000 - val_fn: 8480.0000 - val_accuracy: 0.9282 - val_precision: 0.9282 - val_recall: 0.9282 - val_auc: 0.9756
Epoch 95/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.3177 - tp: 412060.0000 - fp: 60372.0000 - tn: 412060.0000 - fn: 60372.0000 - accura

472432/472432 [==============================] - 3s 7us/sample - loss: 0.3019 - tp: 414573.0000 - fp: 57859.0000 - tn: 414573.0000 - fn: 57859.0000 - accuracy: 0.8775 - precision: 0.8775 - recall: 0.8775 - auc: 0.9459 - val_loss: 0.2599 - val_tp: 106631.0000 - val_fp: 11477.0000 - val_tn: 106631.0000 - val_fn: 11477.0000 - val_accuracy: 0.9028 - val_precision: 0.9028 - val_recall: 0.9028 - val_auc: 0.9631
Epoch 132/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.3022 - tp: 414956.0000 - fp: 57476.0000 - tn: 414956.0000 - fn: 57476.0000 - accuracy: 0.8783 - precision: 0.8783 - recall: 0.8783 - auc: 0.9464 - val_loss: 0.2577 - val_tp: 106743.0000 - val_fp: 11365.0000 - val_tn: 106743.0000 - val_fn: 11365.0000 - val_accuracy: 0.9038 - val_precision: 0.9038 - val_recall: 0.9038 - val_auc: 0.9622
Epoch 133/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.2998 - tp: 415349.0000 - fp: 57083.0000 - tn: 415349.0000 - fn: 57083.0000 - ac

...

472432/472432 [==============================] - 3s 7us/sample - loss: 0.1905 - tp: 435216.0000 - fp: 37216.0000 - tn: 435216.0000 - fn: 37216.0000 - accuracy: 0.9212 - precision: 0.9212 - recall: 0.9212 - auc: 0.9766 - val_loss: 0.2003 - val_tp: 109995.0000 - val_fp: 8113.0000 - val_tn: 109995.0000 - val_fn: 8113.0000 - val_accuracy: 0.9313 - val_precision: 0.9313 - val_recall: 0.9313 - val_auc: 0.9772
Epoch 1842/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1892 - tp: 435751.0000 - fp: 36681.0000 - tn: 435751.0000 - fn: 36681.0000 - accuracy: 0.9224 - precision: 0.9224 - recall: 0.9224 - auc: 0.9770 - val_loss: 0.1834 - val_tp: 110795.0000 - val_fp: 7313.0000 - val_tn: 110795.0000 - val_fn: 7313.0000 - val_accuracy: 0.9381 - val_precision: 0.9381 - val_recall: 0.9381 - val_auc: 0.9805
Epoch 1843/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1857 - tp: 435791.0000 - fp: 36641.0000 - tn: 435791.0000 - fn: 36641.0000 - accu

472432/472432 [==============================] - 3s 7us/sample - loss: 0.1919 - tp: 435374.0000 - fp: 37058.0000 - tn: 435374.0000 - fn: 37058.0000 - accuracy: 0.9216 - precision: 0.9216 - recall: 0.9216 - auc: 0.9766 - val_loss: 0.1973 - val_tp: 109843.0000 - val_fp: 8265.0000 - val_tn: 109843.0000 - val_fn: 8265.0000 - val_accuracy: 0.9300 - val_precision: 0.9300 - val_recall: 0.9300 - val_auc: 0.9774
Epoch 1880/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1894 - tp: 435934.0000 - fp: 36498.0000 - tn: 435934.0000 - fn: 36498.0000 - accuracy: 0.9227 - precision: 0.9227 - recall: 0.9227 - auc: 0.9772 - val_loss: 0.1726 - val_tp: 111686.0000 - val_fp: 6422.0000 - val_tn: 111686.0000 - val_fn: 6422.0000 - val_accuracy: 0.9456 - val_precision: 0.9456 - val_recall: 0.9456 - val_auc: 0.9823
Epoch 1881/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1880 - tp: 435892.0000 - fp: 36540.0000 - tn: 435892.0000 - fn: 36540.0000 - accu

472432/472432 [==============================] - 3s 7us/sample - loss: 0.1886 - tp: 435818.0000 - fp: 36614.0000 - tn: 435818.0000 - fn: 36614.0000 - accuracy: 0.9225 - precision: 0.9225 - recall: 0.9225 - auc: 0.9771 - val_loss: 0.1537 - val_tp: 112688.0000 - val_fp: 5420.0000 - val_tn: 112688.0000 - val_fn: 5420.0000 - val_accuracy: 0.9541 - val_precision: 0.9541 - val_recall: 0.9541 - val_auc: 0.9856
Epoch 1918/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1870 - tp: 436132.0000 - fp: 36300.0000 - tn: 436132.0000 - fn: 36300.0000 - accuracy: 0.9232 - precision: 0.9232 - recall: 0.9232 - auc: 0.9776 - val_loss: 0.1743 - val_tp: 111658.0000 - val_fp: 6450.0000 - val_tn: 111658.0000 - val_fn: 6450.0000 - val_accuracy: 0.9454 - val_precision: 0.9454 - val_recall: 0.9454 - val_auc: 0.9823
Epoch 1919/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1887 - tp: 435764.0000 - fp: 36668.0000 - tn: 435764.0000 - fn: 36668.0000 - accu

472432/472432 [==============================] - 3s 7us/sample - loss: 0.1866 - tp: 435863.0000 - fp: 36569.0000 - tn: 435863.0000 - fn: 36569.0000 - accuracy: 0.9226 - precision: 0.9226 - recall: 0.9226 - auc: 0.9775 - val_loss: 0.2204 - val_tp: 108411.0000 - val_fp: 9697.0000 - val_tn: 108411.0000 - val_fn: 9697.0000 - val_accuracy: 0.9179 - val_precision: 0.9179 - val_recall: 0.9179 - val_auc: 0.9724
Epoch 1956/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1888 - tp: 436031.0000 - fp: 36401.0000 - tn: 436031.0000 - fn: 36401.0000 - accuracy: 0.9229 - precision: 0.9229 - recall: 0.9229 - auc: 0.9774 - val_loss: 0.1890 - val_tp: 110519.0000 - val_fp: 7589.0000 - val_tn: 110519.0000 - val_fn: 7589.0000 - val_accuracy: 0.9357 - val_precision: 0.9357 - val_recall: 0.9357 - val_auc: 0.9794
Epoch 1957/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1862 - tp: 436169.0000 - fp: 36263.0000 - tn: 436169.0000 - fn: 36263.0000 - accu

472432/472432 [==============================] - 3s 7us/sample - loss: 0.1876 - tp: 435711.0000 - fp: 36721.0000 - tn: 435711.0000 - fn: 36721.0000 - accuracy: 0.9223 - precision: 0.9223 - recall: 0.9223 - auc: 0.9773 - val_loss: 0.1693 - val_tp: 111989.0000 - val_fp: 6119.0000 - val_tn: 111989.0000 - val_fn: 6119.0000 - val_accuracy: 0.9482 - val_precision: 0.9482 - val_recall: 0.9482 - val_auc: 0.9831
Epoch 1994/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1858 - tp: 436258.0000 - fp: 36174.0000 - tn: 436258.0000 - fn: 36174.0000 - accuracy: 0.9234 - precision: 0.9234 - recall: 0.9234 - auc: 0.9777 - val_loss: 0.1991 - val_tp: 109873.0000 - val_fp: 8235.0000 - val_tn: 109873.0000 - val_fn: 8235.0000 - val_accuracy: 0.9303 - val_precision: 0.9303 - val_recall: 0.9303 - val_auc: 0.9772
Epoch 1995/2000
472432/472432 [==============================] - 3s 7us/sample - loss: 0.1871 - tp: 436176.0000 - fp: 36256.0000 - tn: 436176.0000 - fn: 36256.0000 - accu

In [21]:
model_clf_proba.save('model_clf_proba')

In [22]:
import pickle
pickle.dump([X_train_alibi, X_val_alibi, X_val_dice, y_val], open('validation_dataset.pkl', 'wb'))
pickle.dump([cat_columns, cat_object_columns, cat_vars_ord, numerical_columns, integer_columns, features, enc], open('encoder_data.pkl', 'wb'))